In [15]:
import sys
sys.path.insert(0, "../scripts/")

from scdir import find, get_text_batch, get_text
import pandas as pd
import requests

from bs4 import BeautifulSoup
import unicodedata
import numpy as np

## Получение списка идентификоторв PII по заданному поиску 

In [7]:
search_results = []
for res in find('(lifespan OR health) AND ("natural gas")', wait_time=1, on_page_count=100):
    search_results += res

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

RESULTS COUNT: 3021


 27%|██████████████████████▏                                                            | 8/30 [00:06<00:18,  1.17it/s]

Retrying... 429


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:25<00:00,  1.19it/s]


> Фильтрация сломанных данных

In [24]:
search_results = [i for i in search_results if len(i) == 3]

In [50]:
articles = pd.DataFrame(search_results, columns=("pii", "title", "date"))
articles = articles[articles['title'].apply(lambda x: 'abstracts' not in x.lower())]
articles.index = pd.RangeIndex(len(articles))
articles

,pii,title,date
0,S2666765721000478,Survey of Airborne Organic Compounds in Reside...,2021-06-13T00:00:00.000Z
1,S0269749119376717,Natural gas leaks and tree death: A first-look...,2020-04-03T00:00:00.000Z
2,S2352484721000202,Experimental optimization of engine performanc...,2021-01-21T00:00:00.000Z
3,S2213138820300308,Environmental life cycle assessment of glyceri...,2020-10-09T00:00:00.000Z
4,S2352484720312312,Comparative cost assessment of sustainable ene...,2020-07-23T00:00:00.000Z
...,...,...,...
2904,S1383586621006699,Advanced configurations for post-combustion CO...,2021-05-13T00:00:00.000Z
2905,S0021925818693585,The Depression of the Incorporation of Sulfur ...,2021-01-04T00:00:00.000Z
2906,S1750583619306395,CO<ce:inf loc=post>2</ce:inf> capture adsorben...,2020-03-03T00:00:00.000Z
2907,S0360128516300442,Fuel consumption and CO<ce:inf loc=post>2</ce:...,2017-02-21T00:00:00.000Z


### Получение полных текстов статей

In [27]:
texts = {}
for link, text in get_text_batch(piis=articles['pii'], n_threads=3):
    texts[link] = text

100%|████████████████████████████████████████████████████████████████████████████| 2909/2909 [1:03:12<00:00,  1.30s/it]


In [51]:
articles = articles.merge(pd.DataFrame(texts.items(), columns=("pii", "text")), on='pii')

In [52]:
articles

,pii,title,date,text
0,S2666765721000478,Survey of Airborne Organic Compounds in Reside...,2021-06-13T00:00:00.000Z,Natural gas production and use has increased i...
1,S0269749119376717,Natural gas leaks and tree death: A first-look...,2020-04-03T00:00:00.000Z,"Urban vegetation, in particular tree canopy, i..."
2,S2352484721000202,Experimental optimization of engine performanc...,2021-01-21T00:00:00.000Z,All data required for reproducing this researc...
3,S2213138820300308,Environmental life cycle assessment of glyceri...,2020-10-09T00:00:00.000Z,Glycerine is the usual commercial name in the ...
4,S2352484720312312,Comparative cost assessment of sustainable ene...,2020-07-23T00:00:00.000Z,The energy of natural gas can be transported i...
...,...,...,...,...
2904,S1383586621006699,Advanced configurations for post-combustion CO...,2021-05-13T00:00:00.000Z,Aqueous ammonia-based capture processes are am...
2905,S0021925818693585,The Depression of the Incorporation of Sulfur ...,2021-01-04T00:00:00.000Z,
2906,S1750583619306395,CO<ce:inf loc=post>2</ce:inf> capture adsorben...,2020-03-03T00:00:00.000Z,Because of the increasing environmental impact...
2907,S0360128516300442,Fuel consumption and CO<ce:inf loc=post>2</ce:...,2017-02-21T00:00:00.000Z,Road transport contributes about one-fifth of ...


In [53]:
articles = articles[articles['text'] != '']
articles = articles[articles['text'].apply(lambda x: len(x) <= 92314)]

In [55]:
articles.to_csv("../data/health_ng.csv", index=False)